In [19]:
# import dask.dataframe as dd
# import pandas as pd
# import numpy as np
# import os
# from datetime import datetime, timedelta
# import time
# import json
# import ast
# from tqdm import tqdm
# import requests
# import warnings
# warnings.filterwarnings('ignore') # turn off warnings

# import psycopg2

# connUAT = psycopg2.connect(
#      host='localhost',
#      database='uat',
#      port=5431,
#      user='postgres',
#      password='1234567890'
#   )

# connPROC = psycopg2.connect(
#      host='localhost',
#      database='prod',
#      port=5431,
#      user='postgres',
#      password='1234567890'
#   )

# cursorUAT = connUAT.cursor()
# cursorPROC = connPROC.cursor()

# def selectdf(query,conn):
#     df = pd.read_sql(query, conn)
#     return df
  
# def SQL_push(sql_table_name,python_table,conn,cursor,inplace):
#     python_table.replace([np.nan], [None],inplace=True)
    
#     if inplace == True:
#         try:
#             sql_query = f'truncate table {sql_table_name}'
#             cursor.execute(sql_query)
#             conn.commit()
#         except:
#             pass
            
#     sql = f"INSERT INTO {sql_table_name} VALUES ({','.join(['%s'] * len(python_table.columns))})"

#     data = [tuple(row) for _, row in tqdm(python_table.iterrows(), total=len(python_table), desc="Preparing data")]

#     # Chèn dữ liệu với xử lý lỗi
#     for row in tqdm(data, total=len(data), desc="Inserting rows"):
#         try:
#             cursor.execute(sql, row)
#         except Exception as e:
#             print(f"Lỗi khi chèn dòng {row}: {e}")
#             conn.rollback()  # Hủy giao dịch khi có lỗi, tránh PostgreSQL khóa transaction

#     # Sau khi xong thì commit lại
#     connUAT.commit()
#     print("PUSH DATA: DONE")

# pathfolder = r'F:\Work\Caohoc_2024_2026\PTPM_project\ml-latest'

In [20]:
# sql_check = '''
#             select distinct *
#             from staging.stg_tmdb_json_movie_metadata stjmm 
#             where request_json <> '{"success": false, "status_code": 34, "status_message": "The resource you requested could not be found."}'
#             order by id
#             '''
# movie_json = selectdf(sql_check,connUAT)
# # list_c['id'] = list_c['id'].astype(str)  # Đảm bảo cùng kiểu dữ liệu
# movie_json.sort_values(by='id',inplace=True)
# movie_json

In [21]:
# data_dict_list = []

# for i in tqdm(set(movie_json['id'])):
#     data_str = movie_json[movie_json['id']==i]['request_json'].values[0]
#     data = f'''{data_str}'''
#     data_dict = json.loads(data)
#     data_dict_list.append(data_dict)
    
# movie_metadata = pd.DataFrame(data_dict_list)
# movie_metadata.sort_values(by='id',inplace=True)
# movie_metadata

In [22]:
# movie_metadata.columns.to_list()

In [23]:
# movie_metadata2 = movie_metadata[['id','title','original_title','original_language','release_date','status','overview','tagline','adult','popularity','homepage','poster_path','runtime','budget','revenue','vote_average','vote_count']]
# movie_metadata2.head(5)

In [24]:
# movie_metadata2.columns.to_list()

In [25]:
# movie_metadata[['id','belongs_to_collection']]

### sử dụng:
- belongs_to_collection: dùng id để lọc những film liên quan lẫn nhau
- Dữ liệu có 126 movieid ko có mã tbdmid
- 1071 movieid có mã tbdmid những ko có dữ liệu trên server 
    - code: tbdm movies_info[movies_info['tmdbId'].isin(list(fall_m['id']))]


In [26]:
tmdb_data = json.loads(sql_json[sql_json['id']==11]['request_json'].values[0])
# tmdb_df = pd.DataFrame(tmdb_data)
tmdb_data

NameError: name 'sql_json' is not defined

In [ ]:
# tmdb_data = json.loads(movie_metadata[movie_metadata['id']==1891]['request_json'].values[0])
# # tmdb_df = pd.DataFrame(tmdb_data)
# # tmdb_data

In [ ]:
[tmdb_data['production_countries'][0]['name'],tmdb_data['spoken_languages'][0]['english_name']
 ,tmdb_data['production_companies'][0]['name']
 ,tmdb_data['overview'],tmdb_data['status'],tmdb_data['release_date']
]

In [ ]:
movies_links[movies_links['tmdbId'].isnull()]

In [35]:
# movies = pd.read_csv(os.path.join(pathfolder,'movies.csv'))
links = pd.read_csv(os.path.join(pathfolder,'links.csv'))
links = links[links['tmdbId'].isnull()==False]
links.sort_values(by='tmdbId',inplace=True)
links


,movieId,imdbId,tmdbId
4366,4470,94675,2.0
12656,61724,92149,3.0
17,18,113101,5.0
474,479,107286,6.0
257,260,76759,11.0
...,...,...,...
86365,288447,28212579,1144338.0
86458,288733,23469212,1145358.0
86466,288753,6810618,1146246.0
86513,288913,17519798,1146249.0


In [ ]:
# # DONE
# movies = pd.read_csv(os.path.join(pathfolder,'movies.csv'))
# movies1 = movies[['movieId','genres']]
# max_col = [len(i) for i in list(movies1['genres'].str.split('|'))]
# movies_genres = pd.DataFrame()

# for i in range(max(max_col)):
#     data = movies.copy()
#     data['genres'] = data['genres'].str.split('|',expand=True)[i]
#     data = data[data['genres'].isnull()==False]
#     movies_genres = pd.concat([movies_genres,data])

# movies_genres.sort_values(by='movieId',ascending=True,inplace=True)
# del movies_genres['title']
# movies_genres

,movieId,genres
0,1,Adventure
0,1,Fantasy
0,1,Animation
0,1,Comedy
0,1,Children
...,...,...
86533,288971,Horror
86534,288975,Documentary
86535,288977,Thriller
86535,288977,Crime


In [43]:
# DONE -- CSV
moives_tags = pd.read_csv(os.path.join(pathfolder,'tags.csv'))
moives_tags['date_tag'] = pd.to_datetime(moives_tags['timestamp'],unit='s')
moives_tags['tag'] = moives_tags['tag'].str.lower().str.strip()
moives_tags['tag'] = moives_tags['tag'].replace('',np.nan)
moives_tags.rename(columns={'tag':'keywords'},inplace=True)
moives_tags['keyid'] = moives_tags['userId'].astype('int').astype('str') + moives_tags['movieId'].astype('int').astype('str') + moives_tags['timestamp'].astype('int').astype('str')
moives_tags = moives_tags[['keyid','userId','movieId','keywords','date_tag']]
moives_tags

,keyid,userId,movieId,keywords,date_tag
0,102601430666558,10,260,good vs evil,2015-05-03 15:22:38
1,102601430666505,10,260,harrison ford,2015-05-03 15:21:45
2,102601430666538,10,260,sci-fi,2015-05-03 15:22:18
3,1412211311600756,14,1221,al pacino,2011-07-25 13:32:36
4,1412211311600746,14,1221,mafia,2011-07-25 13:32:26
...,...,...,...,...,...
2328310,3309231765991507547491,330923,176599,politically correct,2017-10-09 11:11:31
2328311,33093333171351279384,330933,3317,coming of age,2012-10-26 19:23:04
2328312,33093333171351279389,330933,3317,sexuality,2012-10-26 19:23:09
2328313,33094757821154110902,330947,5782,not luc besson,2006-07-28 18:21:42


In [37]:
links[links['movieId']==11]

,movieId,imdbId,tmdbId
10,11,112346,9087.0


In [38]:
links[links['tmdbId']==11]


,movieId,imdbId,tmdbId
257,260,76759,11.0


In [40]:
list(set(moives_tags[moives_tags['movieId']==260]['tag']))

['buena para los frikis',
 'tech',
 'mythology',
 'avoid',
 'nerd',
 'loss of arm',
 'escapist',
 'classic movie',
 'astromech droid',
 'aerial combat',
 'fantasy, adventure',
 'marksman h combat remote',
 'imperial star destroyer',
 'you must see it',
 'all-time great, revolutionary',
 'classic, space epic',
 'mystic warriors',
 'intense',
 'hologram',
 'groundbreaking',
 'memories, going to the theater to watch this huge movie as a kid and having my mind blown by everything in the movie.',
 'forest gump',
 'good vs evil',
 'star was',
 'good for the family',
 'based on hidden fortress',
 'stop motion animation',
 'spielberg/lucas',
 'tale',
 'evil empire',
 'princess heroine',
 'clever names',
 'bad',
 'easy going story',
 'oscar (best effects - visual effects)',
 'astounding',
 'aesthetic',
 'spaceflight',
 'farmer',
 'emblematic',
 'heroic mission',
 'rebel',
 'mystery',
 'space battle',
 'good vs evil sci-fi',
 'good characters',
 'robot',
 'fictional',
 'good acting',
 'poetic',


In [28]:
moives_tags_agg = moives_tags[['movieId','tag']].drop_duplicates()
moives_tags_agg = moives_tags_agg.groupby('movieId').agg(keywords=('tag',list)).reset_index()
moives_tags_agg['keywords'] = moives_tags_agg['keywords'].astype('str')
moives_tags_agg

,movieId,keywords
0,1,"['animation', 'friendship', 'toys', 'disney', ..."
1,2,"['animals', 'based on a book', 'fantasy', 'mag..."
2,3,"['sequel', 'moldy', 'old', 'old age', 'old men..."
3,4,"['characters', 'chick flick', 'girl movie', 'r..."
4,5,"['family', 'pregnancy', 'wedding', '4th wall',..."
...,...,...
53447,288765,"['post-apocalyptic', 'survival', 'tw suicide',..."
53448,288779,['don camillo series']
53449,288849,"['addiction', 'animation', 'short film']"
53450,288937,['anime']


In [ ]:
SQL_push(
         sql_table_name='staging.stg_keywords_list',
         python_table=moives_tags_agg,
         con=connUAT,
         cursor=cursorUAT,
         inplace= True
        )

In [ ]:
# test = moives_tags[moives_tags['movieId']==260][['movieId','tag']]
# test.drop_duplicates(inplace=True)
# test[test['tag'].str.contains('jedi')]

,movieId,tag
21775,260,jedi
136929,260,a boy grows into a jedi
525395,260,i wanna be a jedi
1060916,260,jedi master
1060917,260,jedi mind trick
1060918,260,jedi training
2310866,260,jedi wars


In [ ]:
# moives_tags.isnull().sum()

userId        0
movieId       0
tag          27
date_time     0
dtype: int64

In [ ]:
SQL_push(
            sql_table_name= 'staging.stg_grouplen_moviestags',
            python_table= moives_tags,
            con= connUAT,
            cursor= cursorUAT,
            inplace= True
        )

In [ ]:
# # DONE
# movies_ratings = pd.read_csv(os.path.join(pathfolder,'ratings.csv'))
# movies_ratings['date_time'] = pd.to_datetime(movies_ratings['timestamp'],unit='s')
# del movies_ratings['timestamp']
# movies_ratings

,userId,movieId,rating,date_time
0,1,1,4.0,2008-11-03 17:52:19
1,1,110,4.0,2008-11-05 06:04:46
2,1,158,4.0,2008-11-03 17:31:43
3,1,260,4.5,2008-11-03 18:00:04
4,1,356,5.0,2008-11-03 17:58:39
...,...,...,...,...
33832157,330975,8340,2.0,2004-08-04 01:34:16
33832158,330975,8493,2.5,2004-08-04 02:15:09
33832159,330975,8622,4.0,2004-08-04 01:09:37
33832160,330975,8665,3.0,2004-08-04 01:09:25


In [ ]:
# movies_ratings.isnull().sum()

userId       0
movieId      0
rating       0
date_time    0
dtype: int64

In [ ]:
SQL_push(
            sql_table_name= 'staging.stg_grouplen_moviesratings',
            python_table= movies_ratings,
            con= connUAT,
            cursor= cursorUAT,
            inplace= True
        )

In [ ]:
# # DONE
# genome_s = pd.read_csv(os.path.join(pathfolder,'genome-scores.csv'))
# genome_s

,movieId,tagId,relevance
0,1,1,0.03200
1,1,2,0.02225
2,1,3,0.07000
3,1,4,0.05900
4,1,5,0.12300
...,...,...,...
18472123,288167,1124,0.09875
18472124,288167,1125,0.02950
18472125,288167,1126,0.02275
18472126,288167,1127,0.11225


In [ ]:
# genome_s.isnull().sum()

movieId      0
tagId        0
relevance    0
dtype: int64

In [ ]:
SQL_push(
            sql_table_name= 'staging.stg_genome_scores',
            python_table= genome_s,
            con= connUAT,
            cursor= cursorUAT,
            inplace= True
        )

In [ ]:
# # DONE
# genome_t = pd.read_csv(os.path.join(pathfolder,'genome-tags.csv'))
# genome_t

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s
...,...,...
1123,1124,writing
1124,1125,wuxia
1125,1126,wwii
1126,1127,zombie


In [ ]:
# genome_t.isnull().sum()

tagId    0
tag      0
dtype: int64

In [ ]:
SQL_push(
            sql_table_name= 'staging.stg_genome_tags',
            python_table= genome_t,
            con= connUAT,
            cursor= cursorUAT,
            inplace= True
        )

In [ ]:
# connUAT.commit()
# connUAT.close()